In [1]:
import numpy as np
from keras.utils import np_utils
from scipy.misc import imresize
from sklearn.preprocessing import OneHotEncoder

seed = 1337
np.random.seed(seed)  # for reproducibility

Using TensorFlow backend.


In [2]:
train = np.load('train.npy')
test = np.load('test.npy')

In [3]:
x_train, y_train = np.invert(train[:, 0]), train[:, 1].reshape(train.shape[0], 1)

In [4]:
enc = OneHotEncoder()
Y_train = enc.fit_transform(y_train).toarray()

In [5]:
size = 64
img_rows = size
img_cols = size

In [6]:
for i in range(len(x_train)):
    x_train[i] = imresize(x_train[i], (img_rows, img_cols), interp='bicubic')
x_train = np.dstack(x_train)
x_train = np.transpose(x_train, axes=(2, 0, 1))
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
X_train = x_train.astype('float32')
X_train -= np.mean(X_train)
X_train /= np.std(X_train)

In [7]:
for i in range(len(test)):
    test[i] = imresize(np.invert(test[i]), (img_rows, img_cols), interp='bicubic')
test = np.dstack(test)
test = np.transpose(test, axes=(2, 0, 1))
test = test.reshape(test.shape[0], img_rows, img_cols, 1).astype("float32")
test -= np.mean(test)
test /= np.std(test)

In [9]:
input_shape = (img_rows, img_cols, 1)
nb_classes = 500
batch_size = 128
nb_epoch = 30

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, PReLU
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import *

model = Sequential()

model.add(Dropout(0.1, input_shape=input_shape))

model.add(Convolution2D(64, 5, 5, border_mode='same'))
model.add(PReLU())
model.add(BatchNormalization(axis=3))

model.add(Convolution2D(64, 5, 5, border_mode='same'))
model.add(PReLU())
model.add(BatchNormalization(axis=3))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 5, 5, border_mode='same'))
model.add(PReLU())
model.add(BatchNormalization(axis=3))

model.add(Convolution2D(128, 5, 5, border_mode='same'))
model.add(PReLU())
model.add(BatchNormalization(axis=3))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 5, 5, border_mode='same'))
model.add(PReLU())
model.add(BatchNormalization(axis=3))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(2000))
model.add(PReLU())
model.add(Dropout(0.5))
model.add(Dense(2000))
model.add(PReLU())
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.summary()
opt = Adadelta()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dropout_7 (Dropout)              (None, 64, 64, 1)     0           dropout_input_2[0][0]            
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 64, 64, 64)    1664        dropout_7[0][0]                  
____________________________________________________________________________________________________
prelu_8 (PReLU)                  (None, 64, 64, 64)    262144      convolution2d_6[0][0]            
____________________________________________________________________________________________________
batchnormalization_6 (BatchNorma (None, 64, 64, 64)    256         prelu_8[0][0]                    
___________________________________________________________________________________________

In [ ]:
model.fit(X_train, Y_train, verbose=0, batch_size=batch_size, nb_epoch=nb_epoch)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1,
                             fill_mode='constant')

model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size), samples_per_epoch=len(X_train), verbose=0,
                    nb_epoch=nb_epoch)

In [ ]:
model.fit(X_train, Y_train, verbose=0, batch_size=96, nb_epoch=10)

In [ ]:
model.evaluate(X_train, Y_train, verbose=0)[1]

In [ ]:
pred_cl = model.predict_classes(test, verbose=0)

In [ ]:
write_frame = ["{0},{1}".format(i + 1, enc.active_features_[pred_cl[i]]) for i in range(len(pred_cl))]
write_frame = ["Id,Category"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))